# Transformers for Classification
In this notebook we  willuse transformers to classify our disaster tweets.

Make sure this notebook is running on a GPU as this will speed up training. Click "Runtime" -> "Change Runtime Type" and make sure a GPU is selected as the Hardware accelerator.

First, lets make sure we have the right libraries installed:

In [ ]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 68.8 MB

## Simple Transformers

There are lots of libraries you can use for transformer models, but the [Huggingface Transformers](https://github.com/huggingface/transformers/) library is the most widly used.

It is a little complicated however, so if all you want to do is train a simple model on a dataset then you can use [SimpleTransformers](https://github.com/ThilinaRajapakse/simpletransformers) which hides away a lot of the complexity.

We'll look at the main transformers library later but here's an example of how you can train a basic model using simpletransformers.

First, we need to load some data. The simple transformers library expects data in a [pandas dataframe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html), so lets use pandas to load a csv file.

The following cell downloads a csv file from the web:

In [ ]:
!rm -rf data
!mkdir -p data
!wget https://github.com/ghomasHudson/text-mining-demos-workshop/raw/main/disaster_tweets.zip -O data/data.zip
!unzip -j data/data.zip -d data
!rm data/data.zip

--2024-01-21 20:24:26--  https://github.com/ghomasHudson/text-mining-demos-workshop/raw/main/disaster_tweets.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ghomasHudson/text-mining-demos-workshop/main/disaster_tweets.zip [following]
--2024-01-21 20:24:26--  https://raw.githubusercontent.com/ghomasHudson/text-mining-demos-workshop/main/disaster_tweets.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 410737 (401K) [application/zip]
Saving to: ‘data/data.zip’

data/data.zip       100%[===================>] 401.11K  --.-KB/s    in 0.02s   

2024-01-21 20:24:27 (20.5 MB/s) - ‘data/data.zi

Now lets load it into a pandas dataframe:

In [ ]:
import pandas as pd
train_df = pd.read_csv("data/train.csv")
eval_df = pd.read_csv("data/test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


Simple transformers requires the dataframe to have columns with the names "text" and "label". This is how simpletransformers will know what to do with the data. The [Simple Transformers Documentation](https://simpletransformers.ai/docs/classification-data-formats/) has a page which describes the format you need.

We already have the tweets in the "text" column so simply rename "target" to "labels":

In [ ]:
train_df = train_df.rename(columns={"target": "labels"})
eval_df = eval_df.rename(columns={"target": "labels"})

train_df.head()

,id,keyword,location,text,labels
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


Now that the data is in the correct format, we can train a model using simple transformers:

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.metrics import f1_score, accuracy_score

model_args = ClassificationArgs()
model_args.train_batch_size = 128
model_args.eval_batch_size = 64
model_args.num_train_epochs = 3
model_args.overwrite_output_dir = True


model = ClassificationModel("roberta", "roberta-base", args=model_args)
model.train_model(train_df, eval_df=eval_df, f1=f1_score, accuracy=accuracy_score)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/7519 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/59 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/59 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/59 [00:00<?, ?it/s]

(177, 0.3855257781885438)

The running loss should go down over time. Lets evaluate the final model:

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df, f1=f1_score, accuracy=accuracy_score)
result

  0%|          | 0/94 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

{'mcc': 0.934245633863949,
 'tp': 54,
 'tn': 37,
 'fp': 2,
 'fn': 1,
 'auroc': 0.9888111888111888,
 'auprc': 0.9941558441558442,
 'f1': 0.972972972972973,
 'accuracy': 0.9680851063829787,
 'eval_loss': 0.0816502757370472}

Lets try out our trained model. Try changing the sentence below to see how the model performs on different inputs:

In [ ]:
predictions, _ = model.predict(["My house is burning down!"])

"Disaster" if predictions[0] else "Not Disaster"

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

'Disaster'

# Additional Exercises

1. Experiment with different training parameter values. Can you improve the performance? See the simpletransformers documentation for a [list of the possible parameters](https://simpletransformers.ai/docs/usage/#configuring-a-simple-transformers-model).
2. Try using a pretrained model: [hkayesh/twitter-disaster-nlp](https://huggingface.co/hkayesh/twitter-disaster-nlp?text=The+woods+are+on+fire)
3. Try other datasets. Either your own or one from the [Huggingface Hub](https://huggingface.co/datasets?task_categories=task_categories:text-classification&sort=trending).
  A good choice is the [Twitter financial news sentiment](https://huggingface.co/datasets/zeroshot/twitter-financial-news-sentiment) dataset. Work out how to download it and convert to the format required for simple transformers.
4. Experiment with other transformer models. We used roberta in the example above but you should be able to swap it for any in [this list](https://simpletransformers.ai/docs/classification-specifics/#supported-model-types).

If you want a dataset to experiment with, you might wanna try the yelp product review dataset:

In [ ]:
!wget https://s3.amazonaws.com/fast-ai-nlp/yelp_review_polarity_csv.tgz -O data/data.tgz
!tar -xvzf data/data.tgz -C data/
!mv data/yelp_review_polarity_csv/* data/
!rm -r data/yelp_review_polarity_csv/
!rm data/data.tgz

Be careful with the labels for this dataset. Heres some code to do that for you:

In [ ]:
train_df = pd.read_csv("data/train.csv", header=None)
eval_df = pd.read_csv("data/test.csv", header=None)

train_df.head()

First, notice that the label column has values 1 and 2. We need them to be 0,1 for binary classification. Lets fix that:

In [ ]:
train_df[0] = (train_df[0] == 2).astype(int)
eval_df[0] = (eval_df[0] == 2).astype(int)

train_df.head()

*Now* lets rename the columns to be "text" and "label" alongside removing line breaks:

In [ ]:
train_df = pd.DataFrame(
      {"text": train_df[1].replace(r"\n", " ", regex=True), "labels": train_df[0]}
)

eval_df = pd.DataFrame(
      {"text": eval_df[1].replace(r"\n", " ", regex=True), "labels": eval_df[0]}
)
train_df.head()

It will take a long time for the model to train with such a large dataset so you might want to cut it down a little:

In [ ]:
train_df = train_df.head(4000)
eval_df = eval_df.head(100)

Now continue adding the training code...